In [ ]:
import json
import fasttext
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objects as go

from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
# from bokeh.plotting import figure, output_file, show
# from bokeh.models import HoverTool, ColumnDataSource
from sklearn.preprocessing import LabelEncoder
from huggingface_hub import hf_hub_download
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [3]:
# Załadowanie danych z pliku word_annotations.json
with open('word_annotations.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Przekształcenie danych w DataFrame
df = pd.DataFrame(data)

print(df.head())


   text_id                                           fragment          label
0     4015  Właśnie wróciła m z motelu Flora w Łodzi , ul ...  Brak etykiety
1     4015           natomiast Panie Właścicielki - KOSZMAR .        Obsługa
2     4015                 Zapłacili śmy wcześniej za pobyt ,  Brak etykiety
3     4015  nie mogli śmy się wycofać i to chyba PANIOM do...        Obsługa
4     4015  za wszelką cenę chciały wyrzucić nas z ogródka...        Obsługa


In [4]:
df_new = df[df["label"] != "Brak etykiety"].copy()

## Word2Vec

In [7]:
word2vec_model = KeyedVectors.load_word2vec_format('nkjp+wiki-forms-all-300-skipg-hs-50.txt.gz', binary=False)

In [8]:
# Funkcja, która zwraca wektor dla słowa
def get_word_vector(word, model):
    try:
        return model[word]
    except KeyError:
        # Jeśli słowo nie jest w modelu, zwracamy wektor zerowy
        return np.zeros(model.vector_size)

# Funkcja, która zwraca wektor dla całego tekstu (uśredniony wektor słów)
def get_text_vector(text, model):
    word_vectors = [get_word_vector(word, model) for word in text.split()]
    
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        # Jeśli tekst jest pusty lub brak wektorów, zwracamy wektor zerowy
        return np.zeros(model.vector_size)

# Tworzymy wektory dla każdego fragmentu tekstu
df_new.loc[:, 'vector'] = df_new['fragment'].apply(lambda x: get_text_vector(x, word2vec_model))

# Sprawdzamy, czy każdy wektor ma ten sam wymiar
print(df_new['vector'].apply(lambda x: x.shape))


1       (300,)
3       (300,)
4       (300,)
6       (300,)
8       (300,)
         ...  
1388    (300,)
1390    (300,)
1392    (300,)
1394    (300,)
1396    (300,)
Name: vector, Length: 599, dtype: object


In [9]:
df_new

,text_id,fragment,label,vector
1,4015,natomiast Panie Właścicielki - KOSZMAR .,Obsługa,"[-0.011928832779328028, 0.024840001327296097, ..."
3,4015,nie mogli śmy się wycofać i to chyba PANIOM do...,Obsługa,"[0.08407300027708213, 0.008693584240972996, -0..."
4,4015,za wszelką cenę chciały wyrzucić nas z ogródka...,Obsługa,"[0.05559050196316093, -0.059623502753674985, -..."
6,4015,"Zrugały nas za to , że zamówili śmy pizzę",Obsługa,"[0.1127023334459712, 0.05186211152209176, -0.1..."
8,4015,Menager przepraszała za bezczelne pracodawczyn...,Obsługa,"[-0.03466400042331467, -0.09817900011936824, -..."
...,...,...,...,...
1388,4364,Napoje dozowane z dystrybutorów wykonane są z ...,Jedzenie,"[-0.04563772813840346, 0.05686454457992857, 0...."
1390,4364,Piwo rozcieńczone wodą,Jedzenie,"[-0.01748966674009959, 0.030957665915290516, -..."
1392,4364,nieprzeszkolona barmanka nie umiała wyregulowa...,Obsługa,"[-0.03786900142828623, 0.05281950036684672, -0..."
1394,4364,Kolejnym problemem jest komunikacja po polsku ...,Obsługa,"[-0.041974400915205476, -0.015470299124717712,..."


In [33]:
# Redukcja wymiarowości za pomocą t-SNE
tsne = TSNE(n_components=2, learning_rate='auto', init='pca', random_state=128)
X = np.vstack(df_new['vector'].values)
y = LabelEncoder().fit_transform(df_new['label'])
tsne_results = tsne.fit_transform(X)
df_new['tsne'] = list(tsne_results)

In [39]:
trace_tsne = go.Scatter(
    x=tsne_results[:,0],
    y=tsne_results[:,1],
    mode='markers',
    showlegend=True,
    marker=dict(
        size=8,
        color=y,
        colorscale='Jet',
    ),
    text=df_new['fragment'],
)

layout = go.Layout(title='t-SNE reduction of word2vec vectors', width=800, height=800)
fig = go.Figure(data=[trace_tsne], layout=layout)
py.iplot(fig)

In [20]:
# Przygotowanie danych do wykresu Bokeh
df_new.loc[:, 'x'] = df_new['tsne'].apply(lambda x: x[0])
df_new.loc[:, 'y'] = df_new['tsne'].apply(lambda x: x[1])
source = ColumnDataSource(df_new)

hover = HoverTool(tooltips=[('Fragment', '@fragment'), ('Label', '@label')])

p = figure(title='t-SNE Word Embeddings', tools=[hover], width=800, height=800)
p.scatter('x', 'y', size=10, source=source)

output_file("word_embeddings.html")
show(p)


In [85]:
# Funkcja do znalezienia k-najbardziej podobnych słów
def most_similar_words(word, df, k=5):
    if word not in df['fragment'].values:
        return []
    
    # Wektor dla zadanego słowa
    word_vector = df.loc[df['fragment'] == word, 'vector'].values[0]
    
    # Podobieństwo kosinusowe dla wszystkich słów
    similarities = cosine_similarity([word_vector], np.vstack(df['vector'].values))[0]
    
    # Połączenie słow z ich podobieństwem
    similar_words = list(zip(df['fragment'], similarities))
    
    # Sortowanie według podobieństwa i wybór top k
    similar_words = sorted(similar_words, key=lambda x: x[1], reverse=True)[1:k+1]
    
    return similar_words

# Przykład dla kilku słów
word_1 = df_new['fragment'].values[0]
similar_words = most_similar_words(word_1, df_new)
print(word_1)
display(pd.DataFrame(similar_words, columns=['fragment', 'similarity']))

word_2 = df_new['fragment'].values[1]
similar_words = most_similar_words(word_2, df_new)
print(word_2)
display(pd.DataFrame(similar_words, columns=['fragment', 'similarity']))

natomiast Panie Właścicielki - KOSZMAR .


,fragment,similarity
0,Panie w recepcji - cudowne i cóż z tego ?,0.559427
1,Panie na recepcji bardzo miłe,0.549812
2,To najlepszy hotel w jakim była m pod względem...,0.545209
3,Panie pracujące w recepcji nie podpisują sie w...,0.499109
4,Panie sprzątające mogły by się bardziej starać...,0.470265


nie mogli śmy się wycofać i to chyba PANIOM dodało odwagi ,


,fragment,similarity
0,"we wszystkim brakowało przypraw , i to nie żeb...",0.755022
1,"Śniadanie też uważam , że za skromne jak za tą...",0.733801
2,"niadanie było proste , choć muszę przyznać , ż...",0.729608
3,"by m nie pomyślał , że da się tak zepsuć frytki",0.709401
4,Jakiś buc ( domniemam że właściciel ) siedzący...,0.708772


## FastText

In [56]:
model = fasttext.load_model(hf_hub_download(repo_id="facebook/fasttext-pl-vectors", filename="model.bin"))

In [57]:
def get_text_vector_fasttext(text, model):
    return model.get_sentence_vector(text)

In [62]:
df_new['fasttext_vector'] = df['fragment'].apply(lambda x: get_text_vector_fasttext(x, model))

In [71]:
df_new[['fragment', 'label', 'fasttext_vector']]

,fragment,label,fasttext_vector
1,natomiast Panie Właścicielki - KOSZMAR .,Obsługa,"[0.0041652485, 0.035627414, 0.017866232, 0.021..."
3,nie mogli śmy się wycofać i to chyba PANIOM do...,Obsługa,"[-0.001226044, -0.0016228049, 0.018858662, -0...."
4,za wszelką cenę chciały wyrzucić nas z ogródka...,Obsługa,"[-0.012786967, 0.020502498, 0.0075052367, 0.00..."
6,"Zrugały nas za to , że zamówili śmy pizzę",Obsługa,"[0.015408729, -0.0038041344, 0.0034508347, -0...."
8,Menager przepraszała za bezczelne pracodawczyn...,Obsługa,"[-0.016231846, -0.0039035508, -0.0020870843, 0..."
...,...,...,...
1388,Napoje dozowane z dystrybutorów wykonane są z ...,Jedzenie,"[-0.010172996, 0.030776422, 0.00966832, 0.0129..."
1390,Piwo rozcieńczone wodą,Jedzenie,"[0.0040252386, 0.031913497, -0.014905117, -0.0..."
1392,nieprzeszkolona barmanka nie umiała wyregulowa...,Obsługa,"[-0.0106102, -0.034817766, 0.012042391, -0.004..."
1394,Kolejnym problemem jest komunikacja po polsku ...,Obsługa,"[-0.052566547, 0.026481584, 0.014810613, 0.040..."


In [65]:
# Redukcja wymiarowości za pomocą t-SNE
tsne = TSNE(n_components=2, learning_rate='auto', init='pca', random_state=128)
X = np.vstack(df_new['fasttext_vector'].values)
tsne_results = tsne.fit_transform(X)
df_new['fasttext-tsne'] = list(tsne_results)

In [66]:
# tsne plot
trace_tsne = go.Scatter(
    x=tsne_results[:,0],
    y=tsne_results[:,1],
    mode='markers',
    showlegend=True,
    marker=dict(
        size=8,
        color=y,
        colorscale='Jet',
    ),
    text=df_new['fragment'],
)

layout = go.Layout(title='t-SNE reduction of fasttext vectors', width=800, height=800)
fig = go.Figure(data=[trace_tsne], layout=layout)
py.iplot(fig)

In [84]:
# Funkcja do znalezienia k-najbardziej podobnych słów
def most_similar_words(word, df, k=5):
    if word not in df['fragment'].values:
        return []
    
    # Wektor dla zadanego słowa
    word_vector = df.loc[df['fragment'] == word, 'fasttext_vector'].values[0]
    
    # Podobieństwo kosinusowe dla wszystkich słów
    similarities = cosine_similarity([word_vector], np.vstack(df['fasttext_vector'].values))[0]
    
    # Połączenie słow z ich podobieństwem
    similar_words = list(zip(df['fragment'], similarities))
    
    # Sortowanie według podobieństwa i wybór top k
    similar_words = sorted(similar_words, key=lambda x: x[1], reverse=True)[1:k+1]
    
    return similar_words

# Przykład dla kilku słów
word_1 = df_new['fragment'].values[0]
similar_words = most_similar_words(word_1, df_new)
print(word_1)
display(pd.DataFrame(similar_words, columns=['fragment', 'similarity']))

word_2 = df_new['fragment'].values[1]
similar_words = most_similar_words(word_2, df_new)
print(word_2)
display(pd.DataFrame(similar_words, columns=['fragment', 'similarity']))

natomiast Panie Właścicielki - KOSZMAR .


,fragment,similarity
0,"Myślały , że ustne zapewnienie recepcjonistki ...",0.702838
1,Pani w recepcji ( p . Katarzyna Jajeczna - kie...,0.685552
2,Pan Parkingowy - miły człowiek .,0.682779
3,"Bardzo uprzejma obsługa , szczególnie Panie po...",0.681109
4,śniadania - porażka .,0.674646


nie mogli śmy się wycofać i to chyba PANIOM dodało odwagi ,


,fragment,similarity
0,"we wszystkim brakowało przypraw , i to nie żeb...",0.843715
1,"by m nie pomyślał , że da się tak zepsuć frytki",0.818829
2,"kelner stwierdził , że każdy może tak sobie mó...",0.817648
3,"Śniadanie też uważam , że za skromne jak za tą...",0.807114
4,"Obsługa sprawę olewa i udaje , że nic się nie ...",0.800575
